# Are-MipiA1A2 : Venin

In [1]:
import numpy as np
from numpy import random
import matplotlib.pyplot as plt


#class agent :
#    def __init__(self):
#        self.efficacite= 0.1
#        self.spawn = 0.1
#        self.coagulation = 0.1
#        self.perte = 0.1
#
#globule_rouge = agent()
#globule_blanc = agent()
#plaquette = agent()
#venin = agent()
#coagulation = agent()
#anti_coagulant = agent()



#"""Type : Cell = List[float**6]"""





anti_coagulantefficacite = 0.1
globule_blancefficacite = 0.1
veninefficacite = 0.5
plaquettescoagulation = 2
veninechange=0.5
globule_blancechange=0.5

In [2]:
def init_matrice(N):
    """Int -> Mat[Cell]
    crée une matrice de taille N*N de type : Mat[list[Gr,Gb,Pl,Ve,Co,Ac,boost]]"""
    matrix = np.zeros((N,N,7))
    mat = np.random.uniform(low=0.878, high=0.985, size=(N,N))
    for i in range(N):
        for j in range(N):
            x = mat[i,j]
            matrix[i,j]=(x , (1 - x)*0.023 , (1 - x)*0.977,0,(1 - x)*0.977,0,1)
    return matrix
            

In [3]:
def init_matrice_venin(N,x,y,Ve):
    matrix = np.zeros((N,N,7))
    mat = np.random.uniform(low=0.878, high=0.985, size=(N,N))
    for i in range(N):
        for j in range(N):
            a = mat[i,j]
            matrix[i,j]=(a , (1 - a)*0.023 , (1 - a)*0.977,0,(1 - a)*0.977,0,1)
    matrix[x,y,3]=Ve
    return matrix

In [4]:
def randomlist(N):
    """Int => list[int]*
    retourne une liste de nombres dans un ordre aléatoire len(list)=N """
    list0 = [i for i in range(N)]
    list1=[]
    rlist=[]
    for i in range(n):
        e = np.random.choice(len(list0))
        rlist.append(list0[e])
        list1=[j for j in list0 if j!=list0[e]]
        list0=list1
    return rlist

In [5]:
def selfinteract(mat,N):
    matrix = mat.copy()
    a = [0 for i in range(7)]
    for i in range(N):
        for j in range(N):
            
            a = mat[i,j].copy()
            
            #globule_rouge+venin
            if a[3] > 0:
                a[0] = a[0] - a[3]*veninefficacite
                if a[0]<0:
                    a[0] = 0
            #globule_blanc+venin
            if a[3] > 0:
                a[3] = a[3] - a[1]*globule_blancefficacite
                a[1] = 0
                a[6] = 10
            #plaquettes+plaquettes
            a[4] = a[4] + a[2]*plaquettescoagulation
            a[2] = a[2] - a[2]/plaquettescoagulation
            #coagulation+anti_coagulant
            if a[5]>0:
                a[4] = a[4] - a[5]*anti_coagulantefficacite
                a[5] = a[5] - a[5]/anti_coagulantefficacite
            #globule_blanc+coagulation
            a[4] = a[4] - a[1]*globule_blancefficacite
            if a[4] <0:
                a[4] = 0
            #venin+anti_coagulant
            if a[3] > 0 and a[5]>0:
                a[3] = a[3] - a[5]*anti_coagulantefficacite
                a[5] = a[5] - a[5]/anti_coagulantefficacite
            
            #coagulation
            if a[3]>0:
                a[4]=a[4]+a[3] *veninefficacite
                if a[4]>1:
                    a[4]=1
            
            matrix[i,j] = a.copy()
    return matrix
            

In [6]:
def interact(mat,i,j,k,l):
    matr = mat.copy()
    matr[i,j,3] = mat[i,j,3] + mat[k,l,3]*veninechange - mat[i,j,3]*veninechange
    matr[k,l,3] = mat[k,l,3] + mat[j,k,3]*veninechange - mat[k,l,3]*veninechange
    
    matr[i,j,1] = mat[i,j,1] + mat[k,l,1]*globule_blancechange - mat[i,j,1]*globule_blancechange
    matr[k,l,1] = mat[k,l,1] + mat[i,j,1]*globule_blancechange - mat[k,l,1]*globule_blancechange
    return matr

In [7]:
def inter_1(N,mat):
    
    
    for i in range(N):
        for j in range(N):
            a = mat[i,j]
            if a[3] > 0 :
                k = np.random.choice(4)
                if k == 0 and i > 0:
                    return interact(mat,i,j,i-1,j)
                if k == 1 and j < N:
                    return interact(mat,i,j,i,j+1)
                if k == 2 and i < N :
                    return interact(mat,i,j,i+1,j)
                if k == 3 and j > 0:
                    return interact(mat,i,j,i,j-1)
            
            return mat

In [8]:
def inter_2(N,mat):
    for i in range(N):
        for j in range(N):
            a =  mat[i,j].copy()
            if a[3] > 0 :
                if i>0:
                    mat = interact(mat,i,j,i-1,j)
                if j<N-1:
                    mat = interact(mat,i,j,i,j+1)
                if i<N-1:
                    mat = interact(mat,i,j,i+1,j)
                if j>0:
                    mat = interact(mat,i,j,i,j-1)
    return mat
            

In [9]:
def lis_co(lis,N):
    """list[mat(cell)]*int => list(mat[float])
    convertie une liste de matrice de cellule en liste de matrice de float (Coagulation)"""
    long=len(lis)
    Lr = np.zeros((long,N,N))
    for n in range(long):
        mat = lis[n]
        for i in range(N):
            for j in range(N):
                Lr[n,i,j] = mat[i,j,4]
                
    return Lr
                

In [37]:
def main(N,x,y,Ve,n):
    mat = init_matrice_venin(N,x,y,Ve)
    rlist=np.zeros((n+1,N,N,7))
    rlist[0,:,:,:] = mat.copy()
    for i in range(n):
        mat = inter_2(N,mat)
        mat = selfinteract(mat,N)
        rlist[i+1,:,:,:] = mat.copy()
    return rlist[:,:,:,4]
    

In [38]:
test = main(21,10,10,40,100)

In [39]:
print(test[0])

[[0.05113509 0.06773789 0.05524831 0.10728447 0.02012827 0.0602288
  0.04370714 0.06133943 0.06158973 0.05031771 0.10927528 0.02670285
  0.11888997 0.07791167 0.02272791 0.02800776 0.04796952 0.07012217
  0.0914807  0.02643031 0.05196427]
 [0.02955246 0.07571768 0.0966558  0.06024807 0.10004573 0.09561987
  0.04639371 0.11633757 0.08618824 0.09364217 0.04405562 0.09832819
  0.11138718 0.10769414 0.11680409 0.11506298 0.0527894  0.02223916
  0.03136436 0.09482478 0.06107921]
 [0.07402462 0.10395269 0.06816878 0.07076526 0.03572853 0.02565052
  0.08283629 0.08869514 0.09486873 0.072774   0.07943683 0.07984333
  0.01510598 0.08561381 0.04626556 0.06984058 0.06177759 0.04908803
  0.03248742 0.05847108 0.07583247]
 [0.03727788 0.0205659  0.07199096 0.06340061 0.02404676 0.09226355
  0.06007676 0.11508367 0.09198264 0.03290801 0.04285949 0.07422706
  0.10102834 0.10930796 0.07445265 0.08005125 0.10605099 0.02035073
  0.06575963 0.06860193 0.1075484 ]
 [0.11321826 0.02917409 0.02624398 0.0527

In [40]:
print(test[99])

[[0.46806659 0.55233904 0.49229341 0.75206315 0.32182881 0.52077896
  0.43713977 0.52056641 0.5152567  0.45020228 0.73203521 0.30971824
  0.75521576 0.53914701 0.25378259 0.27043287 0.36191441 0.46633557
  0.5688415  0.24397042 0.3709478 ]
 [0.36437338 0.59475548 0.70136495 0.52425618 0.72414944 0.70371158
  0.45891291 0.80327038 0.64764724 0.67588981 0.41790721 0.67478567
  0.72590401 0.69375201 0.72645818 0.70694688 0.38836442 0.22995247
  0.27110156 0.58457879 0.4168471 ]
 [0.58684309 0.73909427 0.56496729 0.58157882 0.41278309 0.36586522
  0.65139619 0.6793671  0.70556622 0.58751318 0.60891492 0.59662159
  0.25839144 0.59340898 0.38267995 0.48720682 0.43652258 0.36555717
  0.27772071 0.40431216 0.49068161]
 [0.4072524  0.32903685 0.58968776 0.55192812 0.36119038 0.70749098
  0.55167352 0.82683618 0.70980276 0.40895733 0.44666704 0.58700003
  0.70176633 0.72354124 0.53183485 0.54407033 0.6609221  0.22444754
  0.44415386 0.45478446 0.64857419]
 [0.78560839 0.37437959 0.36715823 0.506